In [1]:
import ssid
import quakeio
from ssid import modes
from utilities import Config, extract_channels, list_files, print_modes, mode_statistics

### Data inputs

In [2]:
directory = "uploads/hayward"
pattern  = "*.[zZ][iI][pP]"

In [3]:
data_conf = Config()
data_conf.d = 8 # decimation factor

# # PAINTER RIO DELL TRANSVERSE (CASE 1)
# data_conf.inputs  = [17,3,20]
# data_conf.outputs = [9,7,4]
# # PAINTER RIO DELL LONGITUDINAL (CASE 2)
# data_conf.inputs  = [15,1,18]
# data_conf.outputs = [11]

# # HWY8/MELOLAND TRANSVERSE (CASE 1)
# data_conf.inputs  = [2],
# data_conf.outputs = [5,7,9]
# # HWY8/MELOLAND TRANSVERSE (CASE 2)
# data_conf.inputs  = [11,2,26],
# data_conf.outputs = [5,7,9]
# # HWY8/MELOLAND LONGITUDINAL (CASE 3)
# data_conf.inputs  = [12,4,25],
# data_conf.outputs = [27,8]

# # CROWLEY TRANSVERSE (CASE 1)
# data_conf.inputs = [4]
# data_conf.outputs = [6,7,9]
# # CROWLEY TRANSVERSE (CASE 2)
# data_conf.inputs = [6,4,9]
# data_conf.outputs = [7]
# # CROWLEY TRANSVERSE (CASE 3)
# data_conf.inputs = [4]
# data_conf.outputs = [7]
# # CROWLEY LONGITUDINAL (CASE 4)
# data_conf.inputs = [5]
# data_conf.outputs = [8]

# # RIDGECREST TRANSVERSE (CASE 1)
# data_conf.inputs  = [4]
# data_conf.outputs = [6,7,9]

# # SAMOA TRANSVERSE (CASE 1)
# data_conf.inputs  = [16]
# data_conf.outputs = [10,12]

# # VINCENT TRANSVERSE (CASE 1)
# data_conf.inputs  = [1,9,24]
# data_conf.outputs = [2,5,6]

# # CAPISTRANO TRANSVERSE (CASE 1)
# data_conf.inputs  = [4]
# data_conf.outputs = [10,7]

# HAYWARD TRANSVERSE (CASE 1)
data_conf.inputs  = [2,7,25,18]
data_conf.outputs = [13,15,23,20]
# # HAYWARD LONGITUDINAL
# data_conf.inputs  = [3,6,17],
# data_conf.outputs = [12,14,19]

# # BERNARDINO TRANSVERSE BENT 3 (CASE 1)
# data_conf.inputs  = [6]
# data_conf.outputs = [7,8]
# # BERNARDINO TRANSVERSE BENT 8 (CASE 2)
# data_conf.inputs  = [24]
# data_conf.outputs = [19,20]
# # BERNARDINO LONGITUDINAL BENT 3 (CASE 3)
# data_conf.inputs  = [4]
# data_conf.outputs = [10]
# # BERNARDINO LONGITUDINAL BENT 8 (CASE 4)
# data_conf.inputs  = [22]
# data_conf.outputs = [17,18]


### Method Inputs

### General Parameters
parameter | value
----------|-------
`p ` | number of output channels
`q ` | number of input channels
`nt` | number of timesteps
`dt` | timestep
`d`  | decimation (downsampling) factor

### Specific to Observer Kalman Identification (OKID)
parameter | value
----------|-------
`m`  | number of Markov parameters to compute (at most = nt)

### Specific to Eigensystem Realization Algorithm (ERA)
parameter | value
----------|-------
`no` | number of observability parameters
`nc` | number of controllability parameters
`r ` | reduced model order

### Specific to Data Correlations (DC)
parameter | value
----------|-------
`a`  | (alpha) number of additional block rows in Hankel matrix of correlation matrices
`b`  | (beta) number of additional block columns in Hankel matrix of correlation matrices
`l`  | initial lag
`g`  | lag (gap) between correlations

### Specific to System Realization with Information Matrix (SRIM)
parameter | value
----------|-------
`no` | number of steps used for identification (prediction horizon)

In [4]:
# Set Parameters
conf = Config()
conf.m  = 400
conf.no = 50
conf.nc = 50
conf.r  = 10
conf.a  = 0  
conf.b  = 0  
conf.l  = 10 
conf.g  = 3

In [5]:
files = list_files(directory, pattern)
event_modes = []
for file in files:
    print(file)

    try:
        event = quakeio.read(file, exclusions=["*filter*", "*date*"])
        print("peak acceleration (cm/s/s):", event["peak_accel"])
        inputs, dt = extract_channels(event, data_conf["inputs"], decimate=data_conf["d"])
        outpts, dt = extract_channels(event, data_conf["outputs"], decimate=data_conf["d"])
        realization = ssid.system(method="srim", input=inputs, output=outpts, **conf)
    except Exception as e:
        print(e)
        continue
    ss_modes = modes.modes(realization,dt)
    event_modes.append(list(ss_modes.values()))
    print_modes(ss_modes)

event_frequencies = mode_statistics(event_modes, "freq")
event_frequencies

uploads/hayward/58658_002_20210623_17.15.07.P.zip
peak acceleration (cm/s/s): 908.091
Spectral quantities:
         T(s)      ζ
      3.689      -0.02397 
      0.8932     0.004272 
      0.6255     0.01391  
      0.5596     0.000824 
      0.5017     0.04985  
Mean Period(s): 1.2537084500585707
Standard Dev(s): 1.2247556464400788
uploads/hayward/58658_003_20211002_12.22.00.P.zip
peak acceleration (cm/s/s): 872.494
Spectral quantities:
         T(s)      ζ
      0.8875     0.00109  
      0.7154     -0.01068 
      0.6273     -0.0007619
      0.3701     0.291    
      0.223      0.02074  
Mean Period(s): 0.5646567059880807
Standard Dev(s): 0.2389653102311956
uploads/hayward/58658_004_20210723_08.59.25.P.zip
peak acceleration (cm/s/s): -847.931
Spectral quantities:
         T(s)      ζ
      8.847      0.3848   
      0.713      0.05781  
      0.6324     0.0448   
      0.5589     -0.04767 
      0.4789     -0.01867 
Mean Period(s): 2.2461135600138102
Standard Dev(s): 3.3015903210249

[{'freq': 1.9931609562799026,
  'damp': 0.04984735283880537,
  'modeshape': array([ 0.00283074-0.19939671j,  0.0125782 -0.04497951j,
          0.00057639+0.01782496j, -0.01015896+0.04179574j]),
  'cnd': 1.0824402565355065,
  'distance': -0.6872382686578082},
 {'freq': 4.485050296897778,
  'damp': 0.02074311070989557,
  'modeshape': array([ 0.25048705-0.11714416j, -0.05171456+0.02421922j,
          0.02394083-0.01814916j,  0.02033   -0.02027278j]),
  'cnd': 1.0200335065535826,
  'distance': -0.0337966197506088},
 {'freq': 2.088188228229123,
  'damp': -0.01867133947806887,
  'modeshape': array([-0.02578005-0.00213771j, -0.00666452-0.00261241j,
         -0.00455962+0.01433623j, -0.00053259+0.00040501j]),
  'cnd': 1.0932624306734038,
  'distance': -0.6623195147382402},
 {'freq': 1.5975845711190932,
  'damp': -0.00797207028872942,
  'modeshape': array([-0.00275853+0.02325649j,  0.00100136+0.00052928j,
         -0.00583325+0.0072354j , -0.0055642 -0.00363815j]),
  'cnd': 1.0478646858501364,


### Painter St Rio Dell Hwy 101 89324
![Painter St Rio Dell Hwy 101 89324 Sensors 1](./figures/89324_Page_1.png)
![Painter St Rio Dell Hwy 101 89324 Sensors 2](./figures/89324_Page_2.png)

### Meloland Hwy 8 El Centro 01336
![Meloland Hwy 8 El Centro 01336 Sensors](./figures/01336.png)

### Lake Crowley Hwy 395 54730
![Lake Crowley Hwy 395 54730 Sensors](./figures/54730.png)

### Hayward Hwy 580-238 58658
![Hayward Hwy 580-238 58658 Sensors 1](./figures/58658_Page_1.png)
![Hayward Hwy 580-238 58658 Sensors 2](./figures/58658_Page_2.png)

### San Bernardino I10-215
![San Bernardino I10-215 Sensors](./figures/23631.png)